In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.1 MB/s eta 0:00:00


In [2]:
!pip install torch tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
import os
os.makedirs("/root/.kaggle", exist_ok=True)


In [4]:
!kaggle datasets download -d vijaydevane/blooms-taxonomy-dataset


Dataset URL: https://www.kaggle.com/datasets/vijaydevane/blooms-taxonomy-dataset
License(s): apache-2.0
100% 240k/240k [00:00<00:00, 772kB/s]
100% 240k/240k [00:00<00:00, 771kB/s]


In [6]:
import zipfile

zip_path = "blooms-taxonomy-dataset.zip"
extract_path = "blooms_taxonomy_data"

# Extract ZIP file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

# List the extracted files
import os
os.listdir(extract_path)


['blooms_taxonomy_dataset.csv']

In [7]:
import pandas as pd

df = pd.read_csv("blooms_taxonomy_data/blooms_taxonomy_dataset.csv")
df.head()  # Display the first 5 rows


,Questions,Category
0,About what proportion of the population of the...,BT1
1,Correctly label the brain lobes indicated on t...,BT1
2,Define compound interest.,BT1
3,Define four types of traceability,BT1
4,Define mercantilism.,BT1


In [8]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Preprocessing

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
'''
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = " ".join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

df['clean_text'] = df['Questions'].apply(clean_text)  # Apply cleaning function
df.head()
'''

Converting label to numericals

In [11]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Category'])  # Encode labels
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)

Label Mapping: {'BT1': 0, 'BT2': 1, 'BT3': 2, 'BT4': 3, 'BT5': 4, 'BT6': 5}


Split dataset to train and test

In [12]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Questions'], df['label'], test_size=0.2, random_state=42
)


Tokenization

In [13]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
token_lengths = [len(tokenizer(text)['input_ids']) for text in train_texts]
'''
# Get statistics
print("Max token length:", np.max(token_lengths))
print("Average token length:", np.mean(token_lengths))
print("90th percentile length:", np.percentile(token_lengths, 90))
'''
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Tensor Dataset

In [14]:
import torch

train_labels = torch.tensor(list(train_labels))
val_labels = torch.tensor(list(val_labels))

train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    train_labels
)

val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_encodings['attention_mask']),
    val_labels
)


Dataloader

In [15]:
from torch.utils.data import DataLoader, RandomSampler

# Batch size
batch_size = 32  # Adjust based on GPU memory

# Define train DataLoader
train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=batch_size
)

# Define validation DataLoader
val_dataloader = DataLoader(
    val_dataset,
    sampler=RandomSampler(val_dataset),
    batch_size=batch_size
)

In [16]:
# Load Pretrained BERT model for Classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label_mapping)  # Number of Bloom's Taxonomy categories
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

Optimizer and learning rate

In [19]:
from transformers import get_scheduler

from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)

# Total number of training steps
epochs = 3
total_steps = len(train_dataloader) * epochs

# Learning rate scheduler
scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=total_steps
)


Train model

In [20]:
import torch
import torch.nn as nn
from tqdm import tqdm  # Progress bar
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function (not needed if using Hugging Face Trainer)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(epochs):
    model.train()
    total_loss = 0  # Track loss per epoch

    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}", leave=True)

    for batch in progress_bar:
        batch_inputs, batch_masks, batch_labels = [t.to(device) for t in batch]  # Move to GPU
        optimizer.zero_grad()
        outputs = model(input_ids=batch_inputs, attention_mask=batch_masks, labels=batch_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        progress_bar.set_postfix({"Loss": loss.item()})

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} - Average Training Loss: {avg_train_loss:.4f}")


Epoch 1/3: 100%|██████████| 220/220 [02:12<00:00,  1.66it/s, Loss=0.313]


Epoch 1 - Average Training Loss: 0.9115


Epoch 2/3: 100%|██████████| 220/220 [02:12<00:00,  1.66it/s, Loss=0.0698]


Epoch 2 - Average Training Loss: 0.4712


Epoch 3/3: 100%|██████████| 220/220 [02:12<00:00,  1.66it/s, Loss=0.115]

Epoch 3 - Average Training Loss: 0.3228


In [23]:
    # 🔍 **Validation**
    model.eval()
    all_preds, all_labels = [], []
    total_val_loss = 0
    with torch.no_grad():
        progress_bar = tqdm(val_dataloader, desc="Validating")  # Progress bar
        for batch in progress_bar:
            batch_inputs, batch_masks, batch_labels = [t.to(device) for t in batch]
            outputs = model(input_ids=batch_inputs, attention_mask=batch_masks, labels=batch_labels)
            loss = outputs.loss  # Extract loss
            total_val_loss += loss.item()  # Accumulate loss
            predictions = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(batch_labels.cpu().numpy())
    # ✅ Compute Accuracy, Precision, Recall, and F1-score
    acc = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="weighted")


    print(f"  ✅ Accuracy: {acc:.4%}")
    print(f"  🎯 Precision: {precision:.4f}, 🔄 Recall: {recall:.4f}, 🏆 F1-score: {f1:.4f}")
    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"\n✅ Validation Loss: {avg_val_loss:.4f}")

Validating: 100%|██████████| 55/55 [00:08<00:00,  6.34it/s]

  ✅ Accuracy: 81.5849%
  🎯 Precision: 0.8169, 🔄 Recall: 0.8158, 🏆 F1-score: 0.8153

✅ Validation Loss: 0.5236


Save the model

In [24]:
# Define save path
save_directory = "bert-blooms-taxonomy-classifier"  # Choose a folder name

# Save fine-tuned model
model.save_pretrained(save_directory)

# Save tokenizer
tokenizer.save_pretrained(save_directory)

print(f"✅ Model saved successfully in: {save_directory}")


✅ Model saved successfully in: bert-blooms-taxonomy-classifier


In [25]:
from transformers import BertForSequenceClassification, BertTokenizer

# Load model and tokenizer
model = BertForSequenceClassification.from_pretrained("bert-blooms-taxonomy-classifier")
tokenizer = BertTokenizer.from_pretrained("bert-blooms-taxonomy-classifier")

print("✅ Model loaded successfully!")


✅ Model loaded successfully!


In [27]:
from google.colab import files
uploaded = files.upload()


Saving Testing_Dataset.xlsx to Testing_Dataset (1).xlsx


In [30]:
import pandas as pd

# Load test dataset (Excel format)
test_df = pd.read_excel("Testing_Dataset.xlsx")

# Display a few samples
print(test_df.head())



   Sno                                           Question            Domain  \
0    1                           Define Machine Learning.  Machine Learning   
1    2  Differentiate between Supervised and Unsupervi...  Machine Learning   
2    3  Apply k-NN algorithm on a given dataset and cl...  Machine Learning   
3    4  Compare Decision Trees and Random Forests in t...  Machine Learning   
4    5  Design a recommendation system using collabora...  Machine Learning   

  Blooms Taxonomy level  
0           Remembering  
1         Understanding  
2              Applying  
3             Analyzing  
4              Creating  


Preprocessing

In [31]:
def preprocess_texts(texts, tokenizer, max_length=128):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Tokenize test questions
test_encodings = preprocess_texts(test_df["Question"], tokenizer)
test_inputs = test_encodings["input_ids"].to(device)
test_masks = test_encodings["attention_mask"].to(device)


In [36]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define label mapping
bloom_labels = ["Remember", "Understand", "Apply", "Analyze", "Evaluate", "Create"]

# Ensure tokenization of all test samples
def preprocess_texts(texts, tokenizer, max_length=128):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Tokenize test questions
test_encodings = preprocess_texts(test_df["Question"], tokenizer)

# Run inference in **batches** to avoid truncation
batch_size = 16  # Adjust based on GPU memory
all_predictions = []

for i in range(0, len(test_df), batch_size):
    batch_inputs = test_encodings["input_ids"][i : i + batch_size].to(device)
    batch_masks = test_encodings["attention_mask"][i : i + batch_size].to(device)

    with torch.no_grad():
        outputs = model(input_ids=batch_inputs, attention_mask=batch_masks)
        batch_preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        all_predictions.extend(batch_preds)  # Store all predictions

# Ensure predictions match dataset length
if len(all_predictions) == len(test_df):
    test_df["predicted_category"] = [bloom_labels[p] for p in all_predictions]
else:
    print(f"Warning: Predictions count ({len(all_predictions)}) doesn't match test samples ({len(test_df)})")

# Show sample results
print(test_df[["Question", "predicted_category"]].head(20))



                                             Question predicted_category
0                            Define Machine Learning.           Remember
1   Differentiate between Supervised and Unsupervi...         Understand
2   Apply k-NN algorithm on a given dataset and cl...              Apply
3   Compare Decision Trees and Random Forests in t...            Analyze
4   Design a recommendation system using collabora...             Create
5           Explain the concept of Perplexity in NLP.            Analyze
6   Identify various text preprocessing techniques...           Remember
7   Apply TF-IDF vectorization on a given text dat...              Apply
8   Analyze the impact of stop-word removal in tex...            Analyze
9   Develop a Named Entity Recognition (NER) syste...             Create
10      List the key features of Distributed Systems.           Remember
11                Explain the concept of CAP Theorem.           Remember
12    Implement a simple distributed key-value stor